<a href="https://colab.research.google.com/github/vigneshvalliappan/DRLgames/blob/master/Section_3_3_key_vol_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import relevant libraries

import pandas as pd

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.stem.porter import *

import plotly.express as px
import matplotlib.pyplot as plt

import random
from nltk.corpus import stopwords

from collections import Counter

import math
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# read data
training_data = pd.read_csv('/content/drive/MyDrive/non DL NLP/Project/training_data_set_v3.csv')

training_data = training_data.drop('Unnamed: 0', axis = 1)

training_data

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,EqkT4qw4acnPDE0Ml8Dfog,Xg98DhTV0z4yNiC4LChDtQ,7z2x16M7IuG8KPfMsyVrKA,5,0,0,0,First time visit and it was truly amazing. Eve...,2016-01-30 05:50:59
1,XCRmpw4z8eKLmi1z5wEE4A,5sc03tSvJ10EhK-JGKhdZw,7z2x16M7IuG8KPfMsyVrKA,5,1,0,2,"This is my favorite restaurant in Pittsburgh, ...",2017-05-13 13:30:44
2,-kaduqAoh_hFbtH7fTUCYw,FLuZBFZVoh-lWhBd3S3i0w,7z2x16M7IuG8KPfMsyVrKA,3,0,0,0,Service was lovely. I live in NYC and have ea...,2018-09-03 03:27:35
3,msJzFFkCGmk_JX8o8lXEfA,JAIa7M6Usdt9-LsJDjE8sg,7z2x16M7IuG8KPfMsyVrKA,5,0,0,0,Great food and even better atmosphere. Try at ...,2017-01-27 20:43:47
4,zUujjVzj5X97FEhYMyqR6A,z57vv7gw5cZV1JxHSV6Nvg,7z2x16M7IuG8KPfMsyVrKA,5,0,0,1,Absolute fantastic dinner!! My partner and I c...,2017-07-28 20:40:33
...,...,...,...,...,...,...,...,...,...
60242,w8hkeVnfNu3hPiUT4YHdfg,bLbSNkLggFnqwNNzzq-Ijw,slUn4AINGy3QmDK62YGzoA,5,28,9,20,I'm in the Chinatown area often but there are ...,2019-04-29 16:18:12
60243,Vh8lu4ddh4zXvvjykxOdoQ,9-YU-svmnMfKs_DkcZc0Tg,slUn4AINGy3QmDK62YGzoA,4,4,1,3,Location: Las Vegas Chinatown (Jones & Spring ...,2019-10-29 13:58:46
60244,CZWxvsTA0uy34qVj9zNyEQ,aWDd4vJtY5uOA9euq7VACg,slUn4AINGy3QmDK62YGzoA,4,1,0,0,The place is upscale feeling for a Chinese foo...,2019-11-18 08:46:48
60245,_FhR5ewN_vDbLi6smfIqGA,SyyYQfNQ4a4Lua5RBlggmQ,slUn4AINGy3QmDK62YGzoA,3,0,0,0,Been coming here since it was mamajias. Ummmm...,2019-11-09 02:48:10


In [4]:
# create tokens and associated metrics

tokens = pd.DataFrame()
stop_words = set(stopwords.words('english')) 

tokens['tokenized_minus_stop'] = training_data['text'].apply(lambda x: [tokens.lower() for tokens in word_tokenize(x) if tokens.lower() not in stop_words])
tokens['token_vol_no_stop'] = tokens['tokenized_minus_stop'].apply(lambda y: len(y))

token_volume_per = [np.percentile(tokens['token_vol_no_stop'], i) for i in range(0, 101, 1)]

print(tokens)

print('min_token_volume:', min(tokens['token_vol_no_stop']))
print('max_token_volume:', max(tokens['token_vol_no_stop']))
print()

print('0th percentile of token_vol_no_stop:', token_volume_per[0])
print('10th percentile of token_vol_no_stop:', token_volume_per[10])
print('20th percentile of token_vol_no_stop:', token_volume_per[20])
print('30th percentile of token_vol_no_stop:', token_volume_per[30])
print('40th percentile of token_vol_no_stop:', token_volume_per[40])
print('50th percentile of token_vol_no_stop:', token_volume_per[50])
print('60th percentile of token_vol_no_stop:', token_volume_per[60])
print('70th percentile of token_vol_no_stop:', token_volume_per[70])
print('80th percentile of token_vol_no_stop:', token_volume_per[80])
print('90th percentile of token_vol_no_stop:', token_volume_per[90])
print('100th percentile of token_vol_no_stop:', token_volume_per[100])

                                    tokenized_minus_stop  token_vol_no_stop
0      [first, time, visit, truly, amazing, ., every,...                 26
1      [favorite, restaurant, pittsburgh, ,, anywhere...                 77
2      [service, lovely, ., live, nyc, eaten, spanish...                 64
3      [great, food, even, better, atmosphere, ., try...                 17
4      [absolute, fantastic, dinner, !, !, partner, c...                203
...                                                  ...                ...
60242  ['m, chinatown, area, often, many, restaurants...                104
60243  [location, :, las, vegas, chinatown, (, jones,...                 54
60244  [place, upscale, feeling, chinese, food, joint...                 46
60245  [coming, since, mamajias, ., ummmm, sure, 4.5,...                 72
60246  [amazing, chinese, restaurant, !, 've, needing...                 97

[60247 rows x 2 columns]
min_token_volume: 1
max_token_volume: 685

0th percentile of t

In [5]:
# https://www.codegrepper.com/code-examples/python/find+the+index+of+closest+element+above+and+below+a+value+in+an+array+python
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

In [6]:
# POS tagging + various scoring mechanisms

keywords = set(['service', 'staff', 'waiter', 'waitress', '$', 'price', 'prices', 'cost', 'overpriced', 'expensive', 'cheap', 'inexpensive', 'pricey', 'pricy', 'deal', 'deals', 'specials', 'special', 'pay', 'paid', 'inexpensive', 'location', 'located', 'area', 'neighborhood', 'selection', 'atmosphere', 'lighting', 'experience', 'experienced', 'toilet', 'washroom', 'bathroom', 'crowd', 'crowded', 'wait', 'waited', 'waiting', 'appointment'])
optimal_token_vol_per = 80
norm_div = 1
adj_vol_weight = 0.3

tokens['token_volume_score'] = tokens['token_vol_no_stop'].apply(lambda x: 1 - abs(token_volume_per.index(find_nearest(token_volume_per, x)) - optimal_token_vol_per)/max(optimal_token_vol_per - 0, 100 - optimal_token_vol_per))
tokens['keyword_score_raw'] = tokens['tokenized_minus_stop'].apply(lambda x: len(set([tokens for tokens in x if tokens in keywords]))*norm_div/len(keywords))
tokens['nouns_minus_stop'] = tokens['tokenized_minus_stop'].apply(lambda x: [(tokens, tags) for tokens, tags in nltk.pos_tag(x) if tags.startswith('N')])
tokens['adj_minus_stop'] = tokens['tokenized_minus_stop'].apply(lambda x: [(tokens, tags) for tokens, tags in nltk.pos_tag(x) if tags.startswith('J')])
tokens['adj_volume'] = tokens['adj_minus_stop'].apply(lambda x: len(x))

min_adj_volume = min(tokens['adj_volume'])
max_adj_volume = max(tokens['adj_volume'])
tokens['adj_volume_score'] = tokens['adj_volume'].apply(lambda x: (x - min_adj_volume)/(max_adj_volume - min_adj_volume))

min_keyword_score = min(tokens['keyword_score_raw'])
max_keyword_score = max(tokens['keyword_score_raw'])
tokens['keyword_score_norm'] = tokens['keyword_score_raw'].apply(lambda x: (x - min_keyword_score)/(max_keyword_score - min_keyword_score))

tokens['volume_score'] = tokens['adj_volume_score']*adj_vol_weight + tokens['token_volume_score']*(1 - adj_vol_weight)

token_volume_score_per = [np.percentile(tokens['token_volume_score'], 0), np.percentile(tokens['token_volume_score'], 20), np.percentile(tokens['token_volume_score'], 40), np.percentile(tokens['token_volume_score'], 60), np.percentile(tokens['token_volume_score'], 80), np.percentile(tokens['token_volume_score'], 100)]
print(token_volume_score_per)
print('min_adj_volume:', min_adj_volume)
print('max_adj_volume:', max_adj_volume)

tokens

[0.0, 0.25, 0.5, 0.75, 0.875, 1.0]
min_adj_volume: 0
max_adj_volume: 117


,tokenized_minus_stop,token_vol_no_stop,token_volume_score,keyword_score_raw,nouns_minus_stop,adj_minus_stop,adj_volume,adj_volume_score,keyword_score_norm,volume_score
0,"[first, time, visit, truly, amazing, ., every,...",26,0.2625,0.026316,"[(time, NN), (visit, NN), (plate, NN), (wonder...","[(first, JJ), (amazing, JJ), (single, JJ), (gr...",6,0.051282,0.083333,0.199135
1,"[favorite, restaurant, pittsburgh, ,, anywhere...",77,0.8750,0.078947,"[(restaurant, NN), (pittsburgh, NN), (aspect, ...","[(favorite, JJ), (eaten, JJ), (spent, JJ), (sa...",12,0.102564,0.250000,0.643269
2,"[service, lovely, ., live, nyc, eaten, spanish...",64,0.7875,0.052632,"[(service, NN), (nyc, NN), (style, NN), (tapas...","[(live, JJ), (eaten, JJ), (spanish, JJ), (good...",9,0.076923,0.166667,0.574327
3,"[great, food, even, better, atmosphere, ., try...",17,0.1000,0.026316,"[(food, NN), (item, NN), (section, NN), (split...","[(great, JJ), (least, JJS), (chicken, JJ)]",3,0.025641,0.083333,0.077692
4,"[absolute, fantastic, dinner, !, !, partner, c...",203,0.8000,0.052632,"[(dinner, NN), (partner, NN), (evening, NN), (...","[(absolute, JJ), (fantastic, JJ), (thursday, J...",34,0.290598,0.166667,0.647179
...,...,...,...,...,...,...,...,...,...,...
60242,"['m, chinatown, area, often, many, restaurants...",104,0.9875,0.052632,"[(area, NN), (restaurants, NNS), (place, NN), ...","[(chinatown, JJ), (many, JJ), (good, JJ), (mam...",22,0.188034,0.166667,0.747660
60243,"[location, :, las, vegas, chinatown, (, jones,...",54,0.6875,0.026316,"[(location, NN), (las, NN), (vegas, NNS), (jon...","[(dinner, JJ), (homemade, JJ), (flavorful, JJ)...",6,0.051282,0.083333,0.496635
60244,"[place, upscale, feeling, chinese, food, joint...",46,0.5875,0.000000,"[(place, NN), (food, NN), (joint, NN), (dumpli...","[(upscale, JJ), (chinese, JJ), (soup, JJ), (la...",11,0.094017,0.000000,0.439455
60245,"[coming, since, mamajias, ., ummmm, sure, 4.5,...",72,0.8375,0.000000,"[(mamajias, NN), (stars, NNS), (ownership, NN)...","[(ummmm, JJ), (sure, JJ), (similar, JJ), (cold...",12,0.102564,0.000000,0.617019


In [7]:
# print a particular record to observe various scores

print('tokenized_minus_stop:', tokens.iloc[16608]['tokenized_minus_stop'])
print()

print('token_vol_no_stop:', tokens.iloc[16608]['token_vol_no_stop'])
print('token_volume_score:', tokens.iloc[16608]['token_volume_score'])
print()

print('adj_volume:', tokens.iloc[16608]['adj_volume'])
print('adj_volume_score:', tokens.iloc[16608]['adj_volume_score'])
print()

print('volume_score:', tokens.iloc[16608]['volume_score'])
print()

print('keyword_score_raw:', tokens.iloc[16608]['keyword_score_raw'])
print('keyword_score_norm:', tokens.iloc[16608]['keyword_score_norm'])

tokenized_minus_stop: ['stopped', 'last', 'week', 'conclude', 'graduation', 'celebration', '.', 'place', 'decided', 'upon', ',', 'little', 'skeptical', 'huge', 'fan', 'hookah', 'around', 'smoke', ',', 'went', 'anyway', '.', 'arrived', 'notified', 'show', 'live', 'bands', 'belly', 'dancers', '.', 'cost', '$', '10', '.', 'got', 'drink', '.', 'none', 'us', 'wanted', 'pay', 'due', 'time', '.', 'luckily', ',', 'space', 'available', "n't", 'pay', 'kind', 'enough', 'seat', 'large', 'party', '.', 'place', 'pretty', 'big', 'separate', 'rooms', '.', "n't", 'able', 'go', 'big', 'room', ',', 'due', 'event', ',', 'take', 'peek', 'saw', ',', 'seemed', 'big', 'colorful', '.', 'low', 'lighting', 'sets', 'chill', 'atmosphere', '.', 'plan', 'spend', 'money', ',', "n't", 'happen', '.', 'looked', 'menu', 'said', ',', '``', 'ohh', ',', 'looks', 'good', '...', '!', "''", 'plan', 'spending', 'money', 'went', 'smoke', 'hookah', '.', 'ordered', 'olive', 'mediterranean', 'combo', 'came', 'hummus', ',', 'baba', 

In [8]:
tokens.to_csv('tokens.csv')

In [9]:
# get a subset of tokens dataframe based on whether normalized keyword score is 0 and and volume score is >= 0.5

tokens_subset = tokens.loc[(tokens['keyword_score_norm'] == 0) & (tokens['volume_score'] >= 0.5)]

tokens_subset

,tokenized_minus_stop,token_vol_no_stop,token_volume_score,keyword_score_raw,nouns_minus_stop,adj_minus_stop,adj_volume,adj_volume_score,keyword_score_norm,volume_score
5,"[5, stars, ., best, restaurant, pittsburgh, ,,...",120,0.9375,0.0,"[(stars, NNS), (restaurant, NN), (pittsburgh, ...","[(humble, JJ), (studied, JJ), (spanish, JJ), (...",14,0.119658,0.0,0.692147
32,"[outstanding, ., maybe, best, meal, 've, ever,...",79,0.8875,0.0,"[(meal, NN), (order, NN), (oxtail, NN), (serra...","[(outstanding, JJ), (best, JJS), (much, JJ), (...",9,0.076923,0.0,0.644327
35,"['m, love, morcilla, !, although, unfortunatel...",64,0.7875,0.0,"[(morcilla, NN), (drinks, NNS), (morcilla, NN)...","[(love, JJ), (unfortunately, JJ), (fabulous, J...",7,0.059829,0.0,0.569199
43,"[going, focus, negative, think, could, 5, star...",156,0.8500,0.0,"[(think, NN), (place, NN), (suoerlative, NN), ...","[(negative, JJ), (star, JJ), (better, JJR), (w...",29,0.247863,0.0,0.669359
51,"['m, state, customer, chose, restaurant, dinne...",63,0.7750,0.0,"[(state, NN), (customer, NN), (dinner, NN), (p...","[(restaurant, JJ), (less, JJR), (intense, JJ),...",8,0.068376,0.0,0.563013
...,...,...,...,...,...,...,...,...,...,...
60207,"[came, ces, upon, recommendation, friend, ., g...",59,0.7375,0.0,"[(ces, NNS), (recommendation, NN), (friend, NN...","[(good, JJ), (next, JJ), (good, JJ), (nice, JJ...",10,0.085470,0.0,0.541891
60214,"[great, food, !, 's, wan, na, say, ., opinion,...",60,0.7500,0.0,"[(food, NN), (na, NNS), (opinion, NN), (food, ...","[(great, JJ), (wan, JJ), (open, JJ), (great, J...",12,0.102564,0.0,0.555769
60218,"[give, two, stars, ,, things, edible, ..., sup...",106,0.9750,0.0,"[(stars, NNS), (things, NNS), (super, NN), (or...","[(decided, JJ), (chinese, JJ), (disappointed, ...",21,0.179487,0.0,0.736346
60239,"[save, room, freshly, baked, egg, tarts, !, !,...",90,0.9500,0.0,"[(room, NN), (egg, NN), (tarts, NNS), (week, N...","[(good, JJ), (pepper, JJ), (good, JJ), (repeat...",11,0.094017,0.0,0.693205
